## Atmospheric PSF simulation basics

The schematic below illustrates the method of simulating atmospheric PSFs. One instance in time is represented here.

Two phase screens are shown, with color displaying the phase shift light will accrue when passing through any location in this screen (units are given in number of wavelengths, for 500nm light). Lines of sight to two stars, at opposite sides of the field of view as shown in the inset, are displayed in grey dot-dash lines. The teal columns represent the path through the atmosphere that photons from each of these stars will travel through; the phase fluctuations sampled by these columns will be contribute to the total atmospheric PSFs. 

At each time step of the simulation, two things happen:
1) simulated photons travel through the phase screens to the telescope and are recorded, and

2) the wind "drifts" each phase screen over the telescope aperture by $v*\delta t$ in the direction of wind velocity (shown by the orange arrows on each screen).
This results in instantaneous PSFs which vary in time in a _temporally_ correlated way, and PSFs which are _spatially_ correlated to each other since there will be overlap in the turbulence sampled by various columns. 

<img src="sim_schematic.png" width="600">

## Simplest usage of psf-weather-station

In [ ]:
import psfws
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import galsim
plt.rcParams['font.size']=14

In [ ]:
# set a seed for rng, if desired
seed = 12435

# instantiate a weather station, this is where one can specify data, correlations, etc. Here defaults are used.
ws = psfws.ParameterGenerator(seed=seed)

# if user wants to draw parameters from a random datapoint, use:
params = ws.draw_parameters()

# what's in here?
params.keys()

These keys correspond to the following parameters:
- `h`: altitudes, in km above sea level, of the phase screens to be simulated. The rest of the parameters below are sampled at these altitudes.
- `speed`, `phi`: wind speed (m/s) and direction. `phi` is defined as the meteorological wind direction, i.e. the direction *from* which the blows, in degrees east of north.
- `j`: turbulence integrals J for each phase screen. These correspond to the `r0_weights` input for `galsim.Atmosphere()`.
- `u`, `v`: the components of wind speed in the E, N directions. Equivalent information to the speed and direction above. 
- `t`: temperature profile. Not required for galsim simulations.
- `edges`: the boundaries between each of the layers. Not required for simulations, but convenient for plotting.

To see these parameters plotted and to explore either the default or your own imported data, see the [exploring environmental parameters](./exploring-environmental-parameters.ipynb) notebook.

### Specifying datapoint for which to get parameters:

If user wants to specify a particular datapoint to draw from, use `ws.get_parameters()` instead, which takes an additional argument of a pandas TimeStamp. This value must be in the index of the data; i.e. must be in `ws.data_fa.index`

In [ ]:
pt = ws.data_fa.index[34]
params = ws.get_parameters(pt)

## Running a simple galsim.Atmosphere() simulation with these parameters

The following cells set up a relatively simple atmospheric PSF simulation. In the interest of quick run-time, the parameters below are set up to simulate a 15s exposure on a single LSSTCam raft, with a 4 phase screen atmosphere. 

The simulation is set up for observations at zenith -- we present the more general setup with coordinate system specifics at the end of this notebook.

### Setting up simulation parameters -- modify these as you like

In [ ]:
# random seends
ATMSEED = 2346
PSFSEED = 98345

# simulation parameters
EXPTIME = 15.
NPSF = 750 # how many stars
FOV = 0.7 # in how big an area, in degrees (here, 0.7 ~= area of 1 LSSTCam raft)

# simulation resolution parameters
NPHOT = int(1e5)
SCREENSCALE = 0.1 # in m, ideally should be <= r0

# atmosphere parameters
NSCREEN = 4
SCREENSIZE = 200 # in m

L0 = 25 # outer scale in m
lam = 754.06 # collection wavelength in nm
r0_500 = 0.16 # Fried parameter (in m) as measured at lambda=500nm

### Set up atmospheric phase screens

In [ ]:
# as above, draw parameters from psf-weather-station given a seed and a number of screens.
ws = psfws.ParameterGenerator(seed=ATMSEED)
params = ws.draw_parameters(nl=NSCREEN)

# observatory altitude must be subtracted from altitude list, since galsim takes 0 to be ground.
params['h'] = [p - ws.h0 for p in params['h']]

# define set of atmospheric phase screens
atm = galsim.Atmosphere(r0_500=r0_500,
                        altitude=list(params['h']),
                        L0=[L0]*NSCREEN,
                        r0_weights=list(params['j']),
                        speed=list(params['speed']),
                        direction=[d*galsim.degrees for d in params['phi']],
                        screen_size=SCREENSIZE,
                        screen_scale=SCREENSCALE,
                        rng=galsim.BaseDeviate(ATMSEED))

# telescope aperture: Rubin-like diameter/obscuration
aper = galsim.Aperture(diam=8.36, obscuration=0.61, lam=lam, screen_list=atm)

# r0 at simulation wavelength sets k-modes to include
r0 = r0_500 * (lam/500)**1.2
kcrit = 0.2
kmax = kcrit / r0

# instantiate phase screen list, for photon shooting simulation
atm.instantiate(kmax=kmax, check='phot')

print("done instantiating!")

### Single star
The following cell executes a PSF simulation, using the atmospheric phase screens from above, for a single star. The atmospheric PSF is convolved with a Gaussian "instrument PSF" of FWHM 0.35 arcsec and drawn, with photon-shooting, onto a postage stamp image with LSST pixel size.

In [ ]:
rng = galsim.BaseDeviate(int(PSFSEED))
theta = (0.0*galsim.degrees, 0.0*galsim.degrees)
psf = atm.makePSF(lam, aper=aper, exptime=EXPTIME, theta=theta)
psf = galsim.Convolve(psf, galsim.Gaussian(fwhm=0.35))
img = psf.drawImage(nx=50, ny=50, scale=0.2, method='phot', n_photons=NPHOT, rng=rng)
plt.imshow(img.array, origin='lower', extent=[-50*0.2, 50*0.2, -50*0.2, 50*0.2])
plt.xlabel('arcsec')
plt.ylabel('arcsec');

### LSSTCam raft
Let's extend the above to simulate an entire raft. 

We populate our field of view with randomly placed stars (using a random number generator seeded with `PSFSEED`). We don't generate an image the size of the field of view with all these PSFs; instead, we generate a postage stamp for each star, as above, and save the measured PSF parameters of each along with it's `(theta_x, theta_y)` position.

In [ ]:
# define empty arrays to hold star positions
thxs = np.empty(NPSF, dtype=float)
thys = np.empty(NPSF, dtype=float)
psfPhotSeeds = np.empty(NPSF, dtype=float)

# generate random uniform numbers into these arrays
psfRng = galsim.BaseDeviate(PSFSEED)
ud = galsim.UniformDeviate(psfRng)
ud.generate(thxs)
ud.generate(thys)
ud.generate(psfPhotSeeds)

# center the random values, and scale by the field of view -- 
# these will now be the theta x/y values for the positions of stars on the FoV
thxs -= 0.5
thys -= 0.5
thxs *= FOV
thys *= FOV

psfPhotSeeds *= 2**20
psfPhotSeeds = psfPhotSeeds.astype(np.int64)

Note: the simulation of each postage stamp _can_ be multithreaded for computational efficiency, but the threading below just enables a progressbar. 

In [ ]:
import threading
from IPython.display import display
import ipywidgets as widgets

progress = widgets.IntProgress(value=0, min=0, max=NPSF-1)

# where to save the PSF parameters
outputs = {'thx':thxs, 'thy':thys, 'seed':psfPhotSeeds, 
           'x':[], 'y': [], 'sigma':[], 'e1':[], 'e2':[]}

def sim_psfs(progress):
    """This function loops through each star and simulates the postage stamp as we saw above. Then, the 
    PSF parameters are measured using HSM and saved to the outputs dictionary."""
    for i, (thx, thy, seed) in enumerate(zip(thxs, thys, psfPhotSeeds)):
        rng = galsim.BaseDeviate(int(seed))
        theta = (thx*galsim.degrees, thy*galsim.degrees)
        psf = atm.makePSF(lam, aper=aper, exptime=EXPTIME, theta=theta)
        psf = galsim.Convolve(psf, galsim.Gaussian(fwhm=0.35))
        img = psf.drawImage(nx=50, ny=50, scale=0.2, method='phot', n_photons=NPHOT, rng=rng)
        mom = galsim.hsm.FindAdaptiveMom(img)

        outputs['x'].append(mom.moments_centroid.x)
        outputs['y'].append(mom.moments_centroid.y)
        outputs['sigma'].append(mom.moments_sigma)
        outputs['e1'].append(mom.observed_shape.e1)
        outputs['e2'].append(mom.observed_shape.e2)
        
        progress.value = i
        
thread = threading.Thread(target=sim_psfs, args=(progress,))
display(progress)
thread.start()

### Outputs

We can now plot the PSF parameter maps from the simulation. Below, we show the PSF size (variation from mean) and a whisker plot of PSF ellipticity. The third panel shows the wind vectors for each layer in the simulation; the length and direction of the lines show speed and direction, and the opacity is related to the average strength of optical turbulence at that elevation. 

In [ ]:
f = plt.figure(figsize=(16, 4))
a = []
a.append(plt.subplot(131))
a.append(plt.subplot(132))
a.append(plt.subplot(133, projection='polar'))

dsigma = np.array(outputs['sigma'])-np.mean(outputs['sigma'])
maxsig = np.max(abs(dsigma))
sig = a[0].hexbin(outputs['thx'], outputs['thy'], C=dsigma, gridsize=25, vmax=maxsig, vmin=-maxsig)
a[0].set_aspect('equal')
plt.colorbar(sig, ax=a[0], use_gridspec=True, label=r'$\delta \sigma$')

e = np.hypot(outputs['e1'], outputs['e2'])
beta = 0.5 * np.arctan2(outputs['e2'], outputs['e1'])

qdict = dict(angles='uv', headlength=0, headwidth=0, minlength=0, 
             pivot='middle', color='#41336d', width=0.004)
q = a[1].quiver(outputs['thx'], outputs['thy'], e*np.cos(beta), e*np.sin(beta), scale=1.0, **qdict)
a[1].set_aspect('equal')
a[1].quiverkey(q, 1.2, 0.95, 0.05, 'e = 0.05', labelpos='N')

a[0].set_title('PSF size', fontsize=15)
a[1].set_title('PSF ellipticity', fontsize=15)
[ax.set_xlabel(r'$\theta_x$ ($^{\circ}$)') for ax in a[:1]]
a[0].set_ylabel(r'$\theta_y$ ($^{\circ}$)');
a[1].set_yticklabels([])

j_weights = np.array([j/np.sum(params['j']) for j in params['j']])
j_weights = j_weights**(3/5)
j_weights /= max(j_weights)

angles = np.deg2rad(params['phi'])
for s, p, j, h in zip(params['speed'], angles, j_weights, params['h']):
    a[2].plot([p,p], [0, s], color='#355e8d', lw=4, alpha=j)
    a[2].text(p - np.deg2rad(5), s+2, f'{h:.1f}km', color='darkgrey')
a[2].set_rgrids([10,20],['',''])
a[2].set_theta_zero_location("N")
a[2].set_theta_direction(-1) 
a[2].set_thetagrids([0,45,90,135,180,225,270,315], labels=['N', '','E', '','S', '','W',''])
a[2].grid(alpha=0.25)

a[2].set_title("Wind vectors", fontsize=14)
a[2].text(0.75, 1.0, "darker = more turbulence", color='#355e8d', transform=a[2].transAxes)
a[2].text(0.9,0.9, r'$v_{max}$= '+f"{max(params['speed']):.2g} m/s", color='#355e8d', transform=a[2].transAxes);

### Technical aside: coordinate systems

In the schematic at the top of this notebook, two coordinate systems are shown: the 'unprimed' system of the ground/telescope, and the 'primed' system of the atmospheric phase screens. 
The primed system is rotated from the ground system according to the altitude and azimuth of observation (in the schematic, the telescope points off zenith). This is depicted because the phase screens in Galsim simulations are always _perpendicular_ to the telescope pointing rather than parallel to the ground. 
This requires the user to specify the environmental parameters not in the ground coordinates, but in sky coordinates. 

The coordinate transforms have been implemented for you in `psf-weather-station`; as the following example illustrates, a few things are needed to use this functionality. In the call to `get_parameters`:

- change the `skycoord` kwarg to `True`

- include the alitutde and azimuth of your simulated observation pointing (the `alt` and `az` kwargs)

- include the latitude and longitude of your observatory location (`lat` and `lon` kwargs. Note that the defaults are set to Rubin observatory)

In [ ]:
params = ws.get_parameters(pt)
params_sky = ws.get_parameters(pt, skycoord=True, alt=80, az=70)

# let's display the wind direction, as an example
plt.plot(params['h'], (params['phi']+360)%360, 'o', label='ground')
plt.plot(params_sky['h'], (params_sky['phi']+360)%360, 'o', label='sky')
plt.legend()
plt.xlabel('altitude (km)')
plt.ylabel('wind direction');

The wind directions change slightly, in this example, as do the the altitudes. This is as intended; the altitude of the layers here correspond not to the actual altitude of the turbulence layer, but to the distance light travels through the atmosphere, which increases by a trigonometric factor of the zenith angle. 